---
title: "Neural Network 訓練の加速"
subtitle: "PyTorch について調べたこと"
author: "司馬博文"
date: 8/6/2024
date-modified: 8/10/2024
image: Files/fig-generation3-output-1.png
categories: [Deep, Python]
bibliography: 
    - ../../../assets/mathematics.bib
    - ../../../assets/bib.bib
    - ../../../assets/bib1.bib
csl: ../../../assets/apalike.csl
abstract-title: 概要
abstract: 前稿で DDPM の実装を紹介したが，実際にローカルのマシンで訓練をしてみると２日かかる．これを加速するためのテクニックを調べた．筆者のローカルマシンは M2 Mac mini であるため，CUDA がなく，皮層的な内容に終始している．Apple Silicon 上では，小さなモデルであっても MPS (Metal Performance Shaders) を用いることで５倍以上の高速化が可能であった．
code-fold: false
execute:
    eval: false
---

::: {.hidden}

::: {.content-visible when-format="html"}

A Blog Entry on Bayesian Computation by an Applied Mathematician

$$

\renewcommand{\P}{\operatorname{P}}\newcommand{\E}{\operatorname{E}}
\newcommand{\R}{\mathbb{R}}\newcommand{\F}{\mathcal{F}}
\newcommand{\abs}[1]{\lvert#1\rvert}\newcommand{\Abs}[1]{\left|#1\right|}\newcommand{\ABs}[1]{\biggl|#1\biggr|}\newcommand{\norm}[1]{\|#1\|}\newcommand{\Norm}[1]{\left\|#1\right\|}\newcommand{\NOrm}[1]{\biggl\|#1\biggr\|}\newcommand{\Brace}[1]{\left\{#1\right\}}\newcommand{\BRace}[1]{\biggl\{#1\biggr\}}\newcommand{\paren}[1]{\left(#1\right)}\newcommand{\Paren}[1]{\biggr(#1\biggl)}\newcommand{\brac}[1]{\langle#1\rangle}\newcommand{\Brac}[1]{\left\langle#1\right\rangle}\newcommand{\BRac}[1]{\biggl\langle#1\biggr\rangle}\newcommand{\bra}[1]{\left\langle#1\right|}\newcommand{\ket}[1]{\left|#1\right\rangle}\newcommand{\Square}[1]{\left[#1\right]}\newcommand{\SQuare}[1]{\biggl[#1\biggr]}\newcommand{\rN}{\mathrm{N}}\newcommand{\ov}[1]{\overline{#1}}\newcommand{\un}[1]{\underline{#1}}\newcommand{\wt}[1]{\widetilde{#1}}\newcommand{\wh}[1]{\widehat{#1}}\newcommand{\pp}[2]{\frac{\partial #1}{\partial #2}}\newcommand{\ppp}[3]{\frac{\partial #1}{\partial #2\partial #3}}\newcommand{\dd}[2]{\frac{d #1}{d #2}}\newcommand{\floor}[1]{\lfloor#1\rfloor}\newcommand{\Floor}[1]{\left\lfloor#1\right\rfloor}\newcommand{\ceil}[1]{\lceil#1\rceil}\newcommand{\ocinterval}[1]{(#1]}\newcommand{\cointerval}[1]{[#1)}\newcommand{\COinterval}[1]{\left[#1\right)}\newcommand{\iso}{\overset{\sim}{\to}}



\newcommand{\y}{\b{y}}\newcommand{\mi}{\,|\,}\newcommand{\Mark}{\mathrm{Mark}}
\newcommand{\argmax}{\operatorname*{argmax}}\newcommand{\argmin}{\operatorname*{argmin}}

\newcommand{\pr}{\mathrm{pr}}\newcommand{\Conv}{\operatorname{Conv}}\newcommand{\cU}{\mathcal{U}}
\newcommand{\Map}{\mathrm{Map}}\newcommand{\dom}{\mathrm{Dom}\;}\newcommand{\cod}{\mathrm{Cod}\;}\newcommand{\supp}{\mathrm{supp}\;}
\newcommand{\grad}{\operatorname{grad}}\newcommand{\rot}{\operatorname{rot}}\renewcommand{\div}{\operatorname{div}}\newcommand{\tr}{\operatorname{tr}}\newcommand{\Tr}{\operatorname{Tr}}\newcommand{\KL}{\operatorname{KL}}\newcommand{\JS}{\operatorname{JS}}\newcommand{\ESS}{\operatorname{ESS}}\newcommand{\MSE}{\operatorname{MSE}}\newcommand{\erf}{\operatorname{erf}}\newcommand{\arctanh}{\operatorname{arctanh}}\newcommand{\pl}{\operatorname{pl}}\newcommand{\minimize}{\operatorname{minimize}}\newcommand{\subjectto}{\operatorname{subject to}}\newcommand{\sinc}{\operatorname{sinc}}\newcommand{\Ent}{\operatorname{Ent}}\newcommand{\Polya}{\operatorname{Polya}}\newcommand{\Exp}{\operatorname{Exp}}\newcommand{\codim}{\operatorname{codim}}\newcommand{\sgn}{\operatorname{sgn}}\newcommand{\rank}{\operatorname{rank}}

\newcommand{\vctr}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\vctrr}[3]{\begin{pmatrix}#1\\#2\\#3\end{pmatrix}}\newcommand{\mtrx}[4]{\begin{pmatrix}#1&#2\\#3&#4\end{pmatrix}}\newcommand{\smtrx}[4]{\paren{\begin{smallmatrix}#1&#2\\#3&#4\end{smallmatrix}}}\newcommand{\Ker}{\mathrm{Ker}\;}\newcommand{\Coker}{\mathrm{Coker}\;}\newcommand{\Coim}{\mathrm{Coim}\;}\newcommand{\lcm}{\mathrm{lcm}}\newcommand{\GL}{\mathrm{GL}}\newcommand{\SL}{\mathrm{SL}}\newcommand{\alt}{\mathrm{alt}}

\renewcommand{\Re}{\mathrm{Re}\;}\renewcommand{\Im}{\mathrm{Im}\,}\newcommand{\Gal}{\mathrm{Gal}}\newcommand{\PGL}{\mathrm{PGL}}\newcommand{\PSL}{\mathrm{PSL}}\newcommand{\Log}{\mathrm{Log}\,}\newcommand{\Res}{\mathrm{Res}\,}\newcommand{\on}{\mathrm{on}\;}\newcommand{\hatC}{\widehat{\C}}\newcommand{\hatR}{\hat{\R}}\newcommand{\PV}{\mathrm{P.V.}}\newcommand{\diam}{\mathrm{diam}}\newcommand{\Area}{\mathrm{Area}}\newcommand{\Lap}{\Laplace}\newcommand{\f}{\mathbf{f}}\newcommand{\cR}{\mathcal{R}}\newcommand{\const}{\mathrm{const.}}\newcommand{\Om}{\Omega}\newcommand{\Cinf}{C^\infty}\newcommand{\ep}{\epsilon}\newcommand{\dist}{\mathrm{dist}}\newcommand{\opart}{\o{\partial}}\newcommand{\Length}{\mathrm{Length}}

\newcommand{\cA}{\mathcal{A}}\newcommand{\cO}{\mathcal{O}}\newcommand{\cW}{\mathcal{W}}\renewcommand{\O}{\mathcal{O}}\renewcommand{\S}{\mathcal{S}}\newcommand{\U}{\mathcal{U}}\newcommand{\V}{\mathrm{V}}\newcommand{\N}{\mathbb{N}}\newcommand{\bN}{\mathbb{N}}\newcommand{\C}{\mathrm{C}}\newcommand{\bC}{\mathbb{C}}\newcommand{\Z}{\mathcal{Z}}\newcommand{\Q}{\mathbb{Q}}\newcommand{\bQ}{\mathbb{Q}}\newcommand{\TV}{\mathrm{TV}}\newcommand{\ORD}{\mathrm{ORD}}\newcommand{\Card}{\mathrm{Card}\,}\newcommand{\Top}{\mathrm{Top}}\newcommand{\Disc}{\mathrm{Disc}}\newcommand{\Codisc}{\mathrm{Codisc}}\newcommand{\CoDisc}{\mathrm{CoDisc}}\newcommand{\Ult}{\mathrm{Ult}}\newcommand{\ord}{\mathrm{ord}}\newcommand{\bS}{\mathbb{S}}\newcommand{\PConn}{\mathrm{PConn}}\newcommand{\mult}{\mathrm{mult}}\newcommand{\inv}{\mathrm{inv}}

\newcommand{\Der}{\mathrm{Der}}\newcommand{\osub}{\overset{\mathrm{open}}{\subset}}\newcommand{\osup}{\overset{\mathrm{open}}{\supset}}\newcommand{\al}{\alpha}\newcommand{\K}{\mathbb{K}}\newcommand{\Sp}{\mathrm{Sp}}\newcommand{\g}{\mathfrak{g}}\newcommand{\h}{\mathfrak{h}}\newcommand{\Imm}{\mathrm{Imm}}\newcommand{\Imb}{\mathrm{Imb}}\newcommand{\Gr}{\mathrm{Gr}}

\newcommand{\Ad}{\mathrm{Ad}}\newcommand{\finsupp}{\mathrm{fin\;supp}}\newcommand{\SO}{\mathrm{SO}}\newcommand{\SU}{\mathrm{SU}}\newcommand{\acts}{\curvearrowright}\newcommand{\mono}{\hookrightarrow}\newcommand{\epi}{\twoheadrightarrow}\newcommand{\Stab}{\mathrm{Stab}}\newcommand{\nor}{\mathrm{nor}}\newcommand{\T}{\mathbb{T}}\newcommand{\Aff}{\mathrm{Aff}}\newcommand{\rsup}{\triangleright}\newcommand{\subgrp}{\overset{\mathrm{subgrp}}{\subset}}\newcommand{\Ext}{\mathrm{Ext}}\newcommand{\sbs}{\subset}\newcommand{\sps}{\supset}\newcommand{\In}{\mathrm{in}\;}\newcommand{\Tor}{\mathrm{Tor}}\newcommand{\p}{\b{p}}\newcommand{\q}{\mathfrak{q}}\newcommand{\m}{\mathfrak{m}}\newcommand{\cS}{\mathcal{S}}\newcommand{\Frac}{\mathrm{Frac}\,}\newcommand{\Spec}{\mathrm{Spec}\,}\newcommand{\bA}{\mathbb{A}}\newcommand{\Sym}{\mathrm{Sym}}\newcommand{\Ann}{\mathrm{Ann}}\newcommand{\Her}{\mathrm{Her}}\newcommand{\Bil}{\mathrm{Bil}}\newcommand{\Ses}{\mathrm{Ses}}\newcommand{\FVS}{\mathrm{FVS}}

\newcommand{\Ho}{\mathrm{Ho}}\newcommand{\CW}{\mathrm{CW}}\newcommand{\lc}{\mathrm{lc}}\newcommand{\cg}{\mathrm{cg}}\newcommand{\Fib}{\mathrm{Fib}}\newcommand{\Cyl}{\mathrm{Cyl}}\newcommand{\Ch}{\mathrm{Ch}}
\newcommand{\rP}{\mathrm{P}}\newcommand{\rE}{\mathrm{E}}\newcommand{\e}{\b{e}}\renewcommand{\k}{\b{k}}\newcommand{\Christ}[2]{\begin{Bmatrix}#1\\#2\end{Bmatrix}}\renewcommand{\Vec}[1]{\overrightarrow{\mathrm{#1}}}\newcommand{\hen}[1]{\mathrm{#1}}\renewcommand{\b}[1]{\boldsymbol{#1}}

\newcommand{\Inc}{\mathrm{Inc}}\newcommand{\aInc}{\mathrm{aInc}}\newcommand{\HS}{\mathrm{HS}}\newcommand{\loc}{\mathrm{loc}}\newcommand{\Lh}{\mathrm{L.h.}}\newcommand{\Epi}{\mathrm{Epi}}\newcommand{\slim}{\mathrm{slim}}\newcommand{\Ban}{\mathrm{Ban}}\newcommand{\Hilb}{\mathrm{Hilb}}\newcommand{\Ex}{\mathrm{Ex}}\newcommand{\Co}{\mathrm{Co}}\newcommand{\sa}{\mathrm{sa}}\newcommand{\nnorm}[1]{{\left\vert\kern-0.25ex\left\vert\kern-0.25ex\left\vert #1 \right\vert\kern-0.25ex\right\vert\kern-0.25ex\right\vert}}\newcommand{\dvol}{\mathrm{dvol}}\newcommand{\Sconv}{\mathrm{Sconv}}\newcommand{\I}{\mathcal{I}}\newcommand{\nonunital}{\mathrm{nu}}\newcommand{\cpt}{\mathrm{cpt}}\newcommand{\lcpt}{\mathrm{lcpt}}\newcommand{\com}{\mathrm{com}}\newcommand{\Haus}{\mathrm{Haus}}\newcommand{\proper}{\mathrm{proper}}\newcommand{\infinity}{\mathrm{inf}}\newcommand{\TVS}{\mathrm{TVS}}\newcommand{\ess}{\mathrm{ess}}\newcommand{\ext}{\mathrm{ext}}\newcommand{\Index}{\mathrm{Index}\;}\newcommand{\SSR}{\mathrm{SSR}}\newcommand{\vs}{\mathrm{vs.}}\newcommand{\fM}{\mathfrak{M}}\newcommand{\EDM}{\mathrm{EDM}}\newcommand{\Tw}{\mathrm{Tw}}\newcommand{\fC}{\mathfrak{C}}\newcommand{\bn}{\boldsymbol{n}}\newcommand{\br}{\boldsymbol{r}}\newcommand{\Lam}{\Lambda}\newcommand{\lam}{\lambda}\newcommand{\one}{\mathbf{1}}\newcommand{\dae}{\text{-a.e.}}\newcommand{\das}{\text{-a.s.}}\newcommand{\td}{\text{-}}\newcommand{\RM}{\mathrm{RM}}\newcommand{\BV}{\mathrm{BV}}\newcommand{\normal}{\mathrm{normal}}\newcommand{\lub}{\mathrm{lub}\;}\newcommand{\Graph}{\mathrm{Graph}}\newcommand{\Ascent}{\mathrm{Ascent}}\newcommand{\Descent}{\mathrm{Descent}}\newcommand{\BIL}{\mathrm{BIL}}\newcommand{\fL}{\mathfrak{L}}\newcommand{\De}{\Delta}

\newcommand{\calA}{\mathcal{A}}\newcommand{\calB}{\mathcal{B}}\newcommand{\D}{\mathcal{D}}\newcommand{\Y}{\mathcal{Y}}\newcommand{\calC}{\mathcal{C}}\renewcommand{\ae}{\mathrm{a.e.}\;}\newcommand{\cZ}{\mathcal{Z}}\newcommand{\fF}{\mathfrak{F}}\newcommand{\fI}{\mathfrak{I}}\newcommand{\rV}{\mathrm{V}}\newcommand{\cE}{\mathcal{E}}\newcommand{\sMap}{\sigma\textrm{-}\mathrm{Map}}\newcommand{\cC}{\mathcal{C}}\newcommand{\comp}{\complement}\newcommand{\J}{\mathcal{J}}\newcommand{\sumN}[1]{\sum_{#1\in\N}}\newcommand{\cupN}[1]{\cup_{#1\in\N}}\newcommand{\capN}[1]{\cap_{#1\in\N}}\newcommand{\Sum}[1]{\sum_{#1=1}^\infty}\newcommand{\sumn}{\sum_{n=1}^\infty}\newcommand{\summ}{\sum_{m=1}^\infty}\newcommand{\sumk}{\sum_{k=1}^\infty}\newcommand{\sumi}{\sum_{i=1}^\infty}\newcommand{\sumj}{\sum_{j=1}^\infty}\newcommand{\cupn}{\cup_{n=1}^\infty}\newcommand{\capn}{\cap_{n=1}^\infty}\newcommand{\cupk}{\cup_{k=1}^\infty}\newcommand{\cupi}{\cup_{i=1}^\infty}\newcommand{\cupj}{\cup_{j=1}^\infty}\newcommand{\limn}{\lim_{n\to\infty}}\renewcommand{\L}{\mathcal{L}}\newcommand{\cL}{\mathcal{L}}\newcommand{\Cl}{\mathrm{Cl}}\newcommand{\cN}{\mathcal{N}}\newcommand{\Ae}{\textrm{-a.e.}\;}\renewcommand{\csub}{\overset{\textrm{closed}}{\subset}}\renewcommand{\csup}{\overset{\textrm{closed}}{\supset}}\newcommand{\wB}{\wt{B}}\newcommand{\cG}{\mathcal{G}}\newcommand{\Lip}{\mathrm{Lip}}\newcommand{\AC}{\mathrm{AC}}\newcommand{\Mol}{\mathrm{Mol}}

\newcommand{\Pe}{\mathrm{Pe}}\newcommand{\wR}{\wh{\mathbb{\R}}}\newcommand*{\Laplace}{\mathop{}\!\mathbin\bigtriangleup}\newcommand*{\DAlambert}{\mathop{}\!\mathbin\Box}\newcommand{\bT}{\mathbb{T}}\newcommand{\dx}{\dslash x}\newcommand{\dt}{\dslash t}\newcommand{\ds}{\dslash s}

\newcommand{\round}{\mathrm{round}}\newcommand{\cond}{\mathrm{cond}}\newcommand{\diag}{\mathrm{diag}}
\newcommand{\Adj}{\mathrm{Adj}}\newcommand{\Pf}{\mathrm{Pf}}\newcommand{\Sg}{\mathrm{Sg}}


\newcommand{\aseq}{\overset{\text{a.s.}}{=}}\newcommand{\deq}{\overset{\text{d}}{=}}\newcommand{\cV}{\mathcal{V}}\newcommand{\FM}{\mathrm{FM}}\newcommand{\KR}{\mathrm{KR}}\newcommand{\rba}{\mathrm{rba}}\newcommand{\rca}{\mathrm{rca}}\newcommand{\Prob}{\mathrm{Prob}}\newcommand{\X}{\mathcal{X}}\newcommand{\Meas}{\mathrm{Meas}}\newcommand{\as}{\;\text{a.s.}}\newcommand{\io}{\;\mathrm{i.o.}}\newcommand{\fe}{\;\text{f.e.}}\newcommand{\bF}{\mathbb{F}}\newcommand{\W}{\mathcal{W}}\newcommand{\Pois}{\mathrm{Pois}}\newcommand{\iid}{\text{i.i.d.}}\newcommand{\wconv}{\rightsquigarrow}\newcommand{\Var}{\mathrm{Var}}\newcommand{\xrightarrown}{\xrightarrow{n\to\infty}}\newcommand{\au}{\mathrm{au}}\newcommand{\cT}{\mathcal{T}}\newcommand{\wto}{\overset{\text{w}}{\to}}\newcommand{\dto}{\overset{\text{d}}{\to}}\newcommand{\sto}{\overset{\text{s}}{\to}}\newcommand{\pto}{\overset{\text{p}}{\to}}\newcommand{\mto}{\overset{\text{m}}{\to}}\newcommand{\vto}{\overset{v}{\to}}\newcommand{\Cont}{\mathrm{Cont}}\newcommand{\stably}{\mathrm{stably}}\newcommand{\Np}{\mathbb{N}^+}\newcommand{\oM}{\overline{\mathcal{M}}}\newcommand{\fP}{\mathfrak{P}}\newcommand{\sign}{\mathrm{sign}}
\newcommand{\Borel}{\mathrm{Borel}}\newcommand{\Mid}{\,|\,}\newcommand{\middleMid}{\;\middle|\;}\newcommand{\CP}{\mathrm{CP}}\newcommand{\bD}{\mathbb{D}}\newcommand{\bL}{\mathbb{L}}\newcommand{\fW}{\mathfrak{W}}\newcommand{\DL}{\mathcal{D}\mathcal{L}}\renewcommand{\r}[1]{\mathrm{#1}}\newcommand{\rC}{\mathrm{C}}\newcommand{\qqquad}{\qquad\quad}

\newcommand{\bit}{\mathrm{bit}}

\newcommand{\err}{\mathrm{err}}

\newcommand{\varparallel}{\mathbin{\!/\mkern-5mu/\!}}\newcommand{\Ri}{\mathrm{Ri}}\newcommand{\Cone}{\mathrm{Cone}}\newcommand{\Int}{\mathrm{Int}}

\newcommand{\pre}{\mathrm{pre}}\newcommand{\om}{\omega}


\newcommand{\del}{\partial}
\newcommand{\LHS}{\mathrm{LHS}}\newcommand{\RHS}{\mathrm{RHS}}\newcommand{\bnu}{\boldsymbol{\nu}}\newcommand{\interior}{\mathrm{in}\;}\newcommand{\SH}{\mathrm{SH}}\renewcommand{\v}{\boldsymbol{\nu}}\newcommand{\n}{\mathbf{n}}\newcommand{\ssub}{\Subset}\newcommand{\curl}{\mathrm{curl}}

\newcommand{\Ei}{\mathrm{Ei}}\newcommand{\sn}{\mathrm{sn}}\newcommand{\wgamma}{\widetilde{\gamma}}

\newcommand{\Ens}{\mathrm{Ens}}

\newcommand{\cl}{\mathrm{cl}}\newcommand{\x}{\boldsymbol{x}}

\newcommand{\Do}{\mathrm{Do}}\newcommand{\IV}{\mathrm{IV}}

\newcommand{\AIC}{\mathrm{AIC}}\newcommand{\mrl}{\mathrm{mrl}}\newcommand{\dotx}{\dot{x}}\newcommand{\UMV}{\mathrm{UMV}}\newcommand{\BLU}{\mathrm{BLU}}

\newcommand{\comb}[2]{\begin{pmatrix}#1\\#2\end{pmatrix}}\newcommand{\bP}{\mathbb{P}}\newcommand{\compsub}{\overset{\textrm{cpt}}{\subset}}\newcommand{\lip}{\textrm{lip}}\newcommand{\BL}{\mathrm{BL}}\newcommand{\G}{\mathbb{G}}\newcommand{\NB}{\mathrm{NB}}\newcommand{\oR}{\ov{\R}}\newcommand{\liminfn}{\liminf_{n\to\infty}}\newcommand{\limsupn}{\limsup_{n\to\infty}}\newcommand{\esssup}{\mathrm{ess.sup}}\newcommand{\asto}{\xrightarrow{\as}}\newcommand{\Cov}{\mathrm{Cov}}\newcommand{\cQ}{\mathcal{Q}}\newcommand{\VC}{\mathrm{VC}}\newcommand{\mb}{\mathrm{mb}}\newcommand{\Avar}{\mathrm{Avar}}\newcommand{\bB}{\mathbb{B}}\newcommand{\bW}{\mathbb{W}}\newcommand{\sd}{\mathrm{sd}}\newcommand{\w}[1]{\widehat{#1}}\newcommand{\bZ}{\mathbb{Z}}\newcommand{\Bernoulli}{\mathrm{Ber}}\newcommand{\Ber}{\mathrm{Ber}}\newcommand{\Mult}{\mathrm{Mult}}\newcommand{\BPois}{\mathrm{BPois}}\newcommand{\fraks}{\mathfrak{s}}\newcommand{\frakk}{\mathfrak{k}}\newcommand{\IF}{\mathrm{IF}}\newcommand{\bX}{\boldsymbol{X}}\newcommand{\bx}{\boldsymbol{x}}\newcommand{\indep}{\perp\!\!\!\perp}\newcommand{\IG}{\mathrm{IG}}\newcommand{\Levy}{\mathrm{Levy}}\newcommand{\MP}{\mathrm{MP}}\newcommand{\Hermite}{\mathrm{Hermite}}\newcommand{\Skellam}{\mathrm{Skellam}}\newcommand{\Dirichlet}{\mathrm{Dirichlet}}\renewcommand{\Beta}{\operatorname{Beta}}\newcommand{\bE}{\mathbb{E}}\newcommand{\bG}{\mathbb{G}}\newcommand{\MISE}{\mathrm{MISE}}\newcommand{\logit}{\mathtt{logit}}\newcommand{\expit}{\mathtt{expit}}\newcommand{\cK}{\mathcal{K}}\newcommand{\dl}{\dot{l}}\newcommand{\dotp}{\dot{p}}\newcommand{\wl}{\wt{l}}\newcommand{\Gauss}{\mathrm{Gauss}}\newcommand{\fA}{\mathfrak{A}}\newcommand{\under}{\mathrm{under}\;}\newcommand{\whtheta}{\wh{\theta}}\newcommand{\Em}{\mathrm{Em}}\newcommand{\ztheta}{{\theta_0}}
\newcommand{\rO}{\mathrm{O}}\newcommand{\Bin}{\mathrm{Bin}}\newcommand{\rW}{\mathrm{W}}\newcommand{\rG}{\mathrm{G}}\newcommand{\rB}{\mathrm{B}}\newcommand{\rU}{\mathrm{U}}\newcommand{\HG}{\mathrm{HG}}\newcommand{\GAMMA}{\mathrm{Gamma}}\newcommand{\Cauchy}{\mathrm{Cauchy}}\newcommand{\rt}{\mathrm{t}}\newcommand{\rF}{\mathrm{F}}
\newcommand{\FE}{\mathrm{FE}}\newcommand{\bV}{\boldsymbol{V}}\newcommand{\GLS}{\mathrm{GLS}}\newcommand{\be}{\boldsymbol{e}}\newcommand{\POOL}{\mathrm{POOL}}\newcommand{\GMM}{\mathrm{GMM}}\newcommand{\MM}{\mathrm{MM}}\newcommand{\SSIV}{\mathrm{SSIV}}\newcommand{\JIV}{\mathrm{JIV}}\newcommand{\AR}{\mathrm{AR}}\newcommand{\ILS}{\mathrm{ILS}}\newcommand{\SLS}{\mathrm{SLS}}\newcommand{\LIML}{\mathrm{LIML}}

\newcommand{\Rad}{\mathrm{Rad}}\newcommand{\bY}{\boldsymbol{Y}}\newcommand{\pone}{{(1)}}\newcommand{\ptwo}{{(2)}}\newcommand{\ps}[1]{{(#1)}}\newcommand{\fsub}{\overset{\text{finite}}{\subset}}


\newcommand{\varlim}{\varprojlim}\newcommand{\Hom}{\mathrm{Hom}}\newcommand{\Iso}{\mathrm{Iso}}\newcommand{\Mor}{\mathrm{Mor}}\newcommand{\Isom}{\mathrm{Isom}}\newcommand{\Aut}{\mathrm{Aut}}\newcommand{\End}{\mathrm{End}}\newcommand{\op}{\mathrm{op}}\newcommand{\ev}{\mathrm{ev}}\newcommand{\Ob}{\mathrm{Ob}}\newcommand{\Ar}{\mathrm{Ar}}\newcommand{\Arr}{\mathrm{Arr}}\newcommand{\Set}{\mathrm{Set}}\newcommand{\Grp}{\mathrm{Grp}}\newcommand{\Cat}{\mathrm{Cat}}\newcommand{\Mon}{\mathrm{Mon}}\newcommand{\Ring}{\mathrm{Ring}}\newcommand{\CRing}{\mathrm{CRing}}\newcommand{\Ab}{\mathrm{Ab}}\newcommand{\Pos}{\mathrm{Pos}}\newcommand{\Vect}{\mathrm{Vect}}\newcommand{\FinVect}{\mathrm{FinVect}}\newcommand{\FinSet}{\mathrm{FinSet}}\newcommand{\FinMeas}{\mathrm{FinMeas}}\newcommand{\OmegaAlg}{\Omega\text{-}\mathrm{Alg}}\newcommand{\OmegaEAlg}{(\Omega,E)\text{-}\mathrm{Alg}}\newcommand{\Fun}{\mathrm{Fun}}\newcommand{\Func}{\mathrm{Func}}

\newcommand{\Stoch}{\mathrm{Stoch}}\newcommand{\FinStoch}{\mathrm{FinStoch}}\newcommand{\Copy}{\mathrm{copy}}\newcommand{\Delete}{\mathrm{delete}}
\newcommand{\Bool}{\mathrm{Bool}}\newcommand{\CABool}{\mathrm{CABool}}\newcommand{\CompBoolAlg}{\mathrm{CompBoolAlg}}\newcommand{\BoolAlg}{\mathrm{BoolAlg}}\newcommand{\BoolRng}{\mathrm{BoolRng}}\newcommand{\HeytAlg}{\mathrm{HeytAlg}}\newcommand{\CompHeytAlg}{\mathrm{CompHeytAlg}}\newcommand{\Lat}{\mathrm{Lat}}\newcommand{\CompLat}{\mathrm{CompLat}}\newcommand{\SemiLat}{\mathrm{SemiLat}}\newcommand{\Stone}{\mathrm{Stone}}\newcommand{\Mfd}{\mathrm{Mfd}}\newcommand{\LieAlg}{\mathrm{LieAlg}}
\newcommand{\Op}{\mathrm{Op}}
\newcommand{\Sh}{\mathrm{Sh}}
\newcommand{\Diff}{\mathrm{Diff}}
\newcommand{\B}{\mathcal{B}}\newcommand{\cB}{\mathcal{B}}\newcommand{\Span}{\mathrm{Span}}\newcommand{\Corr}{\mathrm{Corr}}\newcommand{\Decat}{\mathrm{Decat}}\newcommand{\Rep}{\mathrm{Rep}}\newcommand{\Grpd}{\mathrm{Grpd}}\newcommand{\sSet}{\mathrm{sSet}}\newcommand{\Mod}{\mathrm{Mod}}\newcommand{\SmoothMnf}{\mathrm{SmoothMnf}}\newcommand{\coker}{\mathrm{coker}}\newcommand{\Ord}{\mathrm{Ord}}\newcommand{\eq}{\mathrm{eq}}\newcommand{\coeq}{\mathrm{coeq}}\newcommand{\act}{\mathrm{act}}

\newcommand{\apf}{\mathrm{apf}}\newcommand{\opt}{\mathrm{opt}}\newcommand{\IS}{\mathrm{IS}}\newcommand{\IR}{\mathrm{IR}}\newcommand{\iidsim}{\overset{\text{i.i.d.}}{\sim}}\newcommand{\propt}{\,\propto\,}\newcommand{\bM}{\mathbb{M}}\newcommand{\cX}{\mathcal{X}}\newcommand{\cY}{\mathcal{Y}}\newcommand{\cP}{\mathcal{P}}\newcommand{\ola}[1]{\overleftarrow{#1}}

\renewcommand{\iff}{\;\mathrm{iff}\;}
\newcommand{\False}{\mathrm{False}}\newcommand{\True}{\mathrm{True}}
\newcommand{\otherwise}{\mathrm{otherwise}}
\newcommand{\suchthat}{\;\mathrm{s.t.}\;}

\newcommand{\cM}{\mathcal{M}}\newcommand{\M}{\mathbb{M}}\newcommand{\cF}{\mathcal{F}}\newcommand{\cD}{\mathcal{D}}\newcommand{\fX}{\mathfrak{X}}\newcommand{\fY}{\mathfrak{Y}}\newcommand{\fZ}{\mathfrak{Z}}\renewcommand{\H}{\mathcal{H}}\newcommand{\cH}{\mathcal{H}}\newcommand{\fH}{\mathfrak{H}}\newcommand{\bH}{\mathbb{H}}\newcommand{\id}{\mathrm{id}}\newcommand{\A}{\mathcal{A}}
\newcommand{\lmd}{\lambda}
\newcommand{\Lmd}{\Lambda}
\newcommand{\cI}{\mathcal{I}}

\newcommand{\Lrarrow}{\;\;\Leftrightarrow\;\;}
\DeclareMathOperator{\des}{des}
\DeclareMathOperator{\nd}{nd}
\DeclareMathOperator{\dsep}{d-sep}
\DeclareMathOperator{\sep}{sep}
\newcommand{\rLL}{\mathrm{LL}}\newcommand{\HT}{\mathrm{HT}}\newcommand{\PS}{\mathrm{PS}}\newcommand{\rI}{\mathrm{I}}
$$

:::

:::



![[前稿：拡散模型の実装――PyTorch によるハンズオン](DDPM.qmd)](../../../docs/posts/2024/Samplers/DDPM_files/figure-html/fig-encoding-output-1.png){width="70%"}

## 問題点と改善したいこと

データセットの読み込みの段階において，次のコードがある：

```python
kwargs = {'num_workers': 5, 'pin_memory': True, 'prefetch_factor': 2}
train_loader = DataLoader(dataset=train_dataset, batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader  = DataLoader(dataset=test_dataset,  batch_size=inference_batch_size, shuffle=False,  **kwargs)
```

これはデータセット（今回は`MNIST`）を読み込み，iterable 型としての仕様を可能にするためのコードである．

上述の通りのコードだとエポック 18 で `RuntimeError: Shared memory manager connection has timed out` を得たが，`num_workers=0` とするとエラーが発生しなかった．

しかし，`num_workers=0` （デフォルト設定）とすると，デフォルトの単一プロセス処理が実行されるため，並列による高速化の恩恵を受けられない．その結果，１エポック 12 分以上なので，40 時間以上をかける必要が出てきた（寝てる間もディスプレイをオフにするだけでスリープさせず，回し続ける）．

::: {.callout-important appearance="simple" icon="false" title="今回の目標"}

うまく並列処理をするようなコードに書き直すことで，ローカル環境でも１日以内で実行できるようにしたい．

:::

## `DataLoader` の引数について

[`DataLoader` メソッドのドキュメント](https://pytorch.org/docs/stable/data.html) を参照すると，

### `num_workers`

は正整数に設定されると，その数だけ並列に動く 'worker' が起動され，マルチプロセス処理が実行される．

しかし，子プロセスも同等のメモリを占めるため，値が大きすぎるとランタイムエラーが発生する（[issue #13246](https://github.com/pytorch/pytorch/issues/13246#issuecomment-905703662) 参照）．

さらに，この際の並列処理は Python の `multiprocessing` パッケージによるもので，Windows と MacOS では（Unix 系のような `fork()` ではなく） `spawn()` が呼ばれる．これは別のインタープリターを開始するため，コードの大部分を `if __name__ == "__main__":` で囲まない限り，同じコードを何回も実行することとなり，ランタイムエラーが出現することとなる．

### `pin_memeory`

しかし，CUDA 上のテンソルオブジェクトを並列処理で共有することは非推奨であり，その際は自動メモリ固定 (automatic memory pinning) を行う必要がある．

pinned memory とは page-locked メモリとも呼ばれ，通常の pageable メモリより転送速度が速いという．

さて，paging とはなんだろうか？（一旦後回し）

### `prefetch_factor`

は各 `worker` が取ってきてストックしておくバッチの数である．

すなわち，`num_workers * prefetch_factor` だけデータをメモリに読み込んでおくことになる．

## 高速化法

### Google Colab の利用

結局この方法でトレーニングをし，[前稿](DDPM.qmd) を完成させたのであった．

![A100（8/6/2024 時点）](Files/A100.png)

A100 が税込 1,494,000 円であったが，これを利用すると１エポック 22 秒で実行できた．

### `torch.nn.DataParallel` の使用

自分のローカルマシンは CUDA がないため利用できないが，ある場合は `PyTorch` のモジュールで並列処理が可能である．^[[PyTorchでGPUを並列で使えるようにするtorch.nn.DataParallelのメモ](https://qiita.com/m__k/items/87b3b1da15f35321ecf5) などを参照した．]

## `mps` で本当に高速になっているのか？

アップルは [Metal](https://developer.apple.com/jp/metal/) という計算 API を提供しており，これが Apple Silicon で利用できる．


In [ ]:
import torch
DEVICE = torch.device("mps")
train_batch_size = 128
epochs = 1

とし，１エポックにかかる時間を比較する．その他の設定は前節と同様．


In [ ]:
#| echo: false
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

import numpy as np

from tqdm import tqdm
from torchvision.utils import save_image, make_grid
from tqdm import tqdm
from torch.optim import Adam

import math

dataset_path = '~/hirofumi/datasets'

kwargs = {'num_workers': 0, 'pin_memory': True}

dataset = 'MNIST'
img_size = (32, 32, 3)   if dataset == "CIFAR10" else (28, 28, 1) # (width, height, channels)

timestep_embedding_dim = 256
n_layers = 8
hidden_dim = 256
n_timesteps = 1000
beta_minmax=[1e-4, 2e-2]

inference_batch_size = 64
lr = 5e-5

seed = 2024

hidden_dims = [hidden_dim for _ in range(n_layers)]
torch.manual_seed(seed)
np.random.seed(seed)

from torchvision.datasets import MNIST, CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader


transform = transforms.Compose([
        transforms.ToTensor(),
])

import multiprocessing
import os

if __name__ == '__main__':
    
    multiprocessing.set_start_method('spawn')

    if dataset == 'CIFAR10':
        train_dataset = CIFAR10(dataset_path, transform=transform, train=True, download=True)
        test_dataset  = CIFAR10(dataset_path, transform=transform, train=False, download=True)
    else:
        train_dataset = MNIST(dataset_path, transform=transform, train=True, download=True)
        test_dataset  = MNIST(dataset_path, transform=transform, train=False, download=True)

    train_loader = DataLoader(dataset=train_dataset, batch_size=train_batch_size, shuffle=True, **kwargs)
    test_loader  = DataLoader(dataset=test_dataset,  batch_size=inference_batch_size, shuffle=False,  **kwargs)
    
    class SinusoidalPosEmb(nn.Module):
        def __init__(self, dim):
            super().__init__()
            self.dim = dim

        def forward(self, x):
            device = x.device
            half_dim = self.dim // 2
            emb = math.log(10000) / (half_dim - 1)
            emb = torch.exp(torch.arange(half_dim, device=device) * -emb)
            emb = x[:, None] * emb[None, :]
            emb = torch.cat((emb.sin(), emb.cos()), dim=-1)
            return emb

    class ConvBlock(nn.Conv2d):
        """
            Conv2D Block
                Args:
                    x: (N, C_in, H, W)
                Returns:
                    y: (N, C_out, H, W)
        """

        def __init__(self, in_channels, out_channels, kernel_size, activation_fn=None, drop_rate=0.,
                        stride=1, padding='same', dilation=1, groups=1, bias=True, gn=False, gn_groups=8):
            
            if padding == 'same':
                padding = kernel_size // 2 * dilation

            super(ConvBlock, self).__init__(in_channels, out_channels, kernel_size,
                                                stride=stride, padding=padding, dilation=dilation,
                                                groups=groups, bias=bias)

            self.activation_fn = nn.SiLU() if activation_fn else None
            self.group_norm = nn.GroupNorm(gn_groups, out_channels) if gn else None
            
        def forward(self, x, time_embedding=None, residual=False):
            
            if residual:
                # in the paper, diffusion timestep embedding was only applied to residual blocks of U-Net
                x = x + time_embedding
                y = x
                x = super(ConvBlock, self).forward(x)
                y = y + x
            else:
                y = super(ConvBlock, self).forward(x)
            y = self.group_norm(y) if self.group_norm is not None else y
            y = self.activation_fn(y) if self.activation_fn is not None else y
            
            return y
        
    class Denoiser(nn.Module):
        
        def __init__(self, image_resolution, hidden_dims=[256, 256], diffusion_time_embedding_dim = 256, n_times=1000):
            super(Denoiser, self).__init__()
            
            _, _, img_C = image_resolution
            
            self.time_embedding = SinusoidalPosEmb(diffusion_time_embedding_dim)
            
            self.in_project = ConvBlock(img_C, hidden_dims[0], kernel_size=7)
            
            self.time_project = nn.Sequential(
                                    ConvBlock(diffusion_time_embedding_dim, hidden_dims[0], kernel_size=1, activation_fn=True),
                                    ConvBlock(hidden_dims[0], hidden_dims[0], kernel_size=1))
            
            self.convs = nn.ModuleList([ConvBlock(in_channels=hidden_dims[0], out_channels=hidden_dims[0], kernel_size=3)])
            
            for idx in range(1, len(hidden_dims)):
                self.convs.append(ConvBlock(hidden_dims[idx-1], hidden_dims[idx], kernel_size=3, dilation=3**((idx-1)//2),
                                                        activation_fn=True, gn=True, gn_groups=8))                                
                                
            self.out_project = ConvBlock(hidden_dims[-1], out_channels=img_C, kernel_size=3)
            
            
        def forward(self, perturbed_x, diffusion_timestep):
            y = perturbed_x
            
            diffusion_embedding = self.time_embedding(diffusion_timestep)
            diffusion_embedding = self.time_project(diffusion_embedding.unsqueeze(-1).unsqueeze(-2))
            
            y = self.in_project(y)
            
            for i in range(len(self.convs)):
                y = self.convs[i](y, diffusion_embedding, residual = True)
                
            y = self.out_project(y)
                
            return y
        
    model = Denoiser(image_resolution=img_size,
                    hidden_dims=hidden_dims, 
                    diffusion_time_embedding_dim=timestep_embedding_dim, 
                    n_times=n_timesteps).to(DEVICE)

    class Diffusion(nn.Module):
        def __init__(self, model, image_resolution=[32, 32, 3], n_times=1000, beta_minmax=[1e-4, 2e-2], device='cuda'):
        
            super(Diffusion, self).__init__()
        
            self.n_times = n_times
            self.img_H, self.img_W, self.img_C = image_resolution

            self.model = model
            
            # define linear variance schedule(betas)
            beta_1, beta_T = beta_minmax
            betas = torch.linspace(start=beta_1, end=beta_T, steps=n_times).to(device) # follows DDPM paper
            self.sqrt_betas = torch.sqrt(betas)
                                        
            # define alpha for forward diffusion kernel
            self.alphas = 1 - betas
            self.sqrt_alphas = torch.sqrt(self.alphas)
            alpha_bars = torch.cumprod(self.alphas, dim=0)
            self.sqrt_one_minus_alpha_bars = torch.sqrt(1-alpha_bars)
            self.sqrt_alpha_bars = torch.sqrt(alpha_bars)
            
            self.device = device
        
        def extract(self, a, t, x_shape):
            """
                from lucidrains' implementation
                    https://github.com/lucidrains/denoising-diffusion-pytorch/blob/beb2f2d8dd9b4f2bd5be4719f37082fe061ee450/denoising_diffusion_pytorch/denoising_diffusion_pytorch.py#L376
            """
            b, *_ = t.shape
            out = a.gather(-1, t)
            return out.reshape(b, *((1,) * (len(x_shape) - 1)))
        
        def scale_to_minus_one_to_one(self, x):
            # according to the DDPMs paper, normalization seems to be crucial to train reverse process network
            return x * 2 - 1
        
        def reverse_scale_to_zero_to_one(self, x):
            return (x + 1) * 0.5
        
        def make_noisy(self, x_zeros, t): 
            # perturb x_0 into x_t (i.e., take x_0 samples into forward diffusion kernels)
            epsilon = torch.randn_like(x_zeros).to(self.device)
            
            sqrt_alpha_bar = self.extract(self.sqrt_alpha_bars, t, x_zeros.shape)
            sqrt_one_minus_alpha_bar = self.extract(self.sqrt_one_minus_alpha_bars, t, x_zeros.shape)
            
            # Let's make noisy sample!: i.e., Forward process with fixed variance schedule
            #      i.e., sqrt(alpha_bar_t) * x_zero + sqrt(1-alpha_bar_t) * epsilon
            noisy_sample = x_zeros * sqrt_alpha_bar + epsilon * sqrt_one_minus_alpha_bar
        
            return noisy_sample.detach(), epsilon
        
        
        def forward(self, x_zeros):
            x_zeros = self.scale_to_minus_one_to_one(x_zeros)
            
            B, _, _, _ = x_zeros.shape
            
            # (1) randomly choose diffusion time-step
            t = torch.randint(low=0, high=self.n_times, size=(B,)).long().to(self.device)
            
            # (2) forward diffusion process: perturb x_zeros with fixed variance schedule
            perturbed_images, epsilon = self.make_noisy(x_zeros, t)
            
            # (3) predict epsilon(noise) given perturbed data at diffusion-timestep t.
            pred_epsilon = self.model(perturbed_images, t)
            
            return perturbed_images, epsilon, pred_epsilon
        
        
        def denoise_at_t(self, x_t, timestep, t):
            B, _, _, _ = x_t.shape
            if t > 1:
                z = torch.randn_like(x_t).to(self.device)
            else:
                z = torch.zeros_like(x_t).to(self.device)
            
            # at inference, we use predicted noise(epsilon) to restore perturbed data sample.
            epsilon_pred = self.model(x_t, timestep)
            
            alpha = self.extract(self.alphas, timestep, x_t.shape)
            sqrt_alpha = self.extract(self.sqrt_alphas, timestep, x_t.shape)
            sqrt_one_minus_alpha_bar = self.extract(self.sqrt_one_minus_alpha_bars, timestep, x_t.shape)
            sqrt_beta = self.extract(self.sqrt_betas, timestep, x_t.shape)
            
            # denoise at time t, utilizing predicted noise
            x_t_minus_1 = 1 / sqrt_alpha * (x_t - (1-alpha)/sqrt_one_minus_alpha_bar*epsilon_pred) + sqrt_beta*z
            
            return x_t_minus_1.clamp(-1., 1)
                    
        def sample(self, N):
            # start from random noise vector, x_0 (for simplicity, x_T declared as x_t instead of x_T)
            x_t = torch.randn((N, self.img_C, self.img_H, self.img_W)).to(self.device)
            
            # autoregressively denoise from x_T to x_0
            #     i.e., generate image from noise, x_T
            for t in range(self.n_times-1, -1, -1):
                timestep = torch.tensor([t]).repeat_interleave(N, dim=0).long().to(self.device)
                x_t = self.denoise_at_t(x_t, timestep, t)
            
            # denormalize x_0 into 0 ~ 1 ranged values.
            x_0 = self.reverse_scale_to_zero_to_one(x_t)
            
            return x_0
        
        
    diffusion = Diffusion(model, image_resolution=img_size, n_times=n_timesteps, 
                        beta_minmax=beta_minmax, device=DEVICE).to(DEVICE)

    optimizer = Adam(diffusion.parameters(), lr=lr)
    denoising_loss = nn.MSELoss()

In [ ]:
print("Start training DDPMs...")
model.train()

import time

start_time = time.time()

for epoch in range(epochs):
    noise_prediction_loss = 0
    for batch_idx, (x, _) in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()

        x = x.to(DEVICE)
        
        noisy_input, epsilon, pred_epsilon = diffusion(x)
        loss = denoising_loss(pred_epsilon, epsilon)
        
        noise_prediction_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    print("\tEpoch", epoch + 1, "complete!", "\tDenoising Loss: ", noise_prediction_loss / batch_idx)
    
total_time = time.time() - start_time

print("Finish!! Total time: ", total_time)

12:58 であった．一方で，CPU でも訓練してみる．


In [ ]:
DEVICE = torch.device("cpu")
model = Denoiser(image_resolution=img_size,
                    hidden_dims=hidden_dims, 
                    diffusion_time_embedding_dim=timestep_embedding_dim, 
                    n_times=n_timesteps).to(DEVICE)
diffusion = Diffusion(model, image_resolution=img_size, n_times=n_timesteps, beta_minmax=beta_minmax, device=DEVICE).to(DEVICE)

print("Start training DDPMs...")
model.train()

import time

start_time = time.time()

for epoch in range(epochs):
    noise_prediction_loss = 0
    for batch_idx, (x, _) in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()

        x = x.to(DEVICE)
        
        noisy_input, epsilon, pred_epsilon = diffusion(x)
        loss = denoising_loss(pred_epsilon, epsilon)
        
        noise_prediction_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        
    print("\tEpoch", epoch + 1, "complete!", "\tDenoising Loss: ", noise_prediction_loss / batch_idx)
    
total_time = time.time() - start_time

print("Finish!! Total time: ", total_time)

１時間越え！

## 終わりに {.appendix}

あまりに時間がかかるので，本記事は `eval: false` としておく．